# Титаник. Кто выживет?

https://www.kaggle.com/c/titanic/

In [82]:
import numpy as np
import pandas as pd
from collections import Counter

### Данные

In [83]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [84]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [85]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [86]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [87]:
df = pd.concat([train, test],sort=False)

Пол male/female закодируем в 1/0 и удалим переменные, с которыми мы не будем сейчас работать

In [88]:
df["isMale"] = df.Sex.replace({"male": 1, "female":0})
df.drop(["Sex", "Cabin", "Ticket", "Name","Age","Fare", "PassengerId","SibSp","Parch"], axis=1, inplace=True)

In [89]:
df_dummies = pd.get_dummies(df, columns=['Pclass', 'Embarked'])

In [90]:
df_dummies.head(10)

,Survived,isMale,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1,0,0,1,0,0,1
1,1.0,0,1,0,0,1,0,0
2,1.0,0,0,0,1,0,0,1
3,1.0,0,1,0,0,0,0,1
4,0.0,1,0,0,1,0,0,1
5,0.0,1,0,0,1,0,1,0
6,0.0,1,1,0,0,0,0,1
7,0.0,1,0,0,1,0,0,1
8,1.0,0,0,0,1,0,0,1
9,1.0,0,0,1,0,1,0,0


In [91]:
df_dummies=df_dummies[~df_dummies['Survived'].isnull()]

In [92]:
df_dummies.isnull().sum()

Survived      0
isMale        0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [93]:
def HEntropy(l):
    length = len(l)
    cnt = Counter(l)
    
    result = 0
    for value in cnt.values():
        result += value / length * np.log( value / length )
    return -result

def IG(H, df_init, feat1, feat2 ):
    length = len(df_init)
    l_left =  df_init[df_init[feat1]>0][feat2]
    l_right = df_init[df_init[feat1]==0][feat2]
    
    hr = H(df_init[feat1])
    hr_left = H(l_left)
    hr_right = H(l_right)
    result = hr - ( ( len(l_left) / length ) * hr_left ) - ( ( len(l_right) / length ) * hr_right)
    return result    

In [94]:
HEntropy(df_dummies['Survived'])

0.6659119735267652

In [95]:
for col in df_dummies.columns:
    if col=='Survived': continue
    print(col, IG(HEntropy, df_dummies, 'Survived', col))

isMale 0.16785485388318255
Pclass_1 0.1521151923166969
Pclass_2 0.16090210155504903
Pclass_3 0.030525554567072233
Embarked_C 0.1955978170929956
Embarked_Q 0.37174339546604773
Embarked_S 0.08753146569624237
